In [ ]:
%run ./../data/load-dataset.ipynb

In [ ]:
import spacy
import pandas as pd
from tqdm import tqdm
from gensim.utils import deaccent
from gensim.parsing.preprocessing import lower_to_unicode, strip_tags, strip_punctuation, \
                                         strip_non_alphanum, split_alphanum, strip_numeric, strip_short, \
                                         remove_stopwords, stem_text, strip_multiple_whitespaces
from gensim.corpora import Dictionary

In [ ]:
TOKENIZED_DATASET_PATH = f'{PROC_DATA_DIR}/rnn_dataset.pkl'
TOKENS_DICTIONARY_PATH = f'{PROC_DATA_DIR}/rnn_dictionary.dict'

DOCUMENT_FILTERS = [deaccent, lower_to_unicode, strip_tags, strip_numeric, stem_text, strip_multiple_whitespaces]
#DOCUMENT_FILTERS = [deaccent, lower_to_unicode, strip_tags, strip_numeric, strip_multiple_whitespaces]

In [ ]:
def apply_filters(x, filters):
    for f in filters:
        x = f(x)
    return x

In [ ]:
if not Path(TOKENIZED_DATASET_PATH).is_file():
    corpus = df[doc_col].values
    
    logger.info("Preprocessing corpus...")
    df[proc_doc_col] = [apply_filters(doc, filters=DOCUMENT_FILTERS) for doc in tqdm(corpus)]
    logger.info("Tokenizing corpus...")
    
    nlp = spacy.load('en_core_web_sm', exclude=['tok2vec', 'tagger', 'parser', 'senter',
                                                'attribute_ruler', 'lemmatizer', 'ner'])
    df[proc_doc_col] = [[t.text for t in nlp(doc)] for doc in tqdm(df[proc_doc_col])]
    '''
    nlp = spacy.load('en_core_web_lg', exclude=['parser', 'senter', 'ner'])
    #nlp = spacy.load('en_core_web_trf', exclude=['parser', 'ner'])
    df[proc_doc_col] = [[t.lemma_ for t in nlp(doc)] for doc in tqdm(df[proc_doc_col])]
    '''
    
    dictionary = Dictionary(df[proc_doc_col])

    logger.info("Saving tokenized dataset to disk...")
    df.to_pickle(TOKENIZED_DATASET_PATH), dictionary.save(TOKENS_DICTIONARY_PATH)
    logger.info("Tokenized dataset saved to disk.")
else:
    df, dictionary = pd.read_pickle(TOKENIZED_DATASET_PATH), Dictionary.load(TOKENS_DICTIONARY_PATH)

In [ ]:
print(len(dictionary))